# Hierarchical Indexing

지금까지 1차원 및 2차원 data를 저장할 수 있는 Pandas ``Series`` 와 ``DataFrame`` object를 살펴보았다. 

더 높은 차원을 다루는 방법은 2가지 형태가 있다.
- Pandas ``Panel``과 ``Panel4D`` object를 사용하여 3차원, 4차원 데이터를 각각 표현할 수 있다. 
- 혹은 계층적 인덱싱을 통해  ``Series``와 ``DataFrame`` 객체를 더 높은 차원으로 표현할 수 있다. 이를 위해 ``MultiIndex`` object를 사용할 수 있다. 

In [0]:
import pandas as pd
import numpy as np

## A Multiply Indexed Series

1차원``Series``를 이용하여 2차원 데이터를 표현해 보자.


### The bad way

Python tuple을 인덱스 키로 사용할 수 있지만 좋은 방법이 아니다. 

In [104]:
index = [('California', 2000), ('California', 2010),
         ('New York', 2000), ('New York', 2010),
         ('Texas', 2000), ('Texas', 2010)]
populations = [33871648, 37253956,
               18976457, 19378102,
               20851820, 25145561]
pop = pd.Series(populations, index=index)
pop

(California, 2000)    33871648
(California, 2010)    37253956
(New York, 2000)      18976457
(New York, 2010)      19378102
(Texas, 2000)         20851820
(Texas, 2010)         25145561
dtype: int64

In [105]:
pop[('California', 2010):('Texas', 2000)]

(California, 2010)    37253956
(New York, 2000)      18976457
(New York, 2010)      19378102
(Texas, 2000)         20851820
dtype: int64

그러나 2010년 값을 모두 찾는 경우처럼 다루기가 불편하다. 

In [106]:
pop[[i for i in pop.index if i[1] == 2010]]

(California, 2010)    37253956
(New York, 2010)      19378102
(Texas, 2010)         25145561
dtype: int64

### The Better Way: Pandas MultiIndex
Pandas ``MultiIndex``을 이용하면 보다 편리하게 다룰 수 있다. 

In [107]:
index = pd.MultiIndex.from_tuples(index)
index

MultiIndex(levels=[['California', 'New York', 'Texas'], [2000, 2010]],
           codes=[[0, 0, 1, 1, 2, 2], [0, 1, 0, 1, 0, 1]])

새로 만든 ``MultiIndex``는 여러 level과 label을 가진다. 
이를 가지고 pop object를 다시 인덱싱해 볼 수 있다. 

In [108]:
pop = pop.reindex(index)
pop

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

이제 다음과 같이 2010년 데이터를 편리하게 접근할 수 있다. (slicing)

In [109]:
pop[:, 2010]

California    37253956
New York      19378102
Texas         25145561
dtype: int64

### MultiIndex as extra dimension

이러한  계층적 인덱싱을 가진 ``Series`` object는 ``DataFrame``와 개념적으로 동일하다.
그래서 ``unstack()`` 과 ``stack()`` method를 통해 상호 변환이 가능하다. 

In [110]:
pop_df = pop.unstack()
pop_df

,2000,2010
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,25145561


In [111]:
pop_df.stack()

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

따라서 ``DataFrame``에서 계층적 인덱싱이 유용할 것이다.
즉, 다음 예에서처럼 3차원으로 데이터를 저장할 수 있다. 

In [112]:
pop_df = pd.DataFrame({'total': pop,
                       'under18': [9267089, 9284094,
                                   4687374, 4318033,
                                   5906301, 6879014]})
pop_df

total  under18
California 2000  33871648  9267089
           2010  37253956  9284094
New York   2000  18976457  4687374
           2010  19378102  4318033
Texas      2000  20851820  5906301
           2010  25145561  6879014

또한 계층적 인덱싱은 앞서 배운 ufuncs 기능과 여전히 잘 작동한다. 

In [113]:
f_u18 = pop_df['under18'] / pop_df['total']
f_u18.unstack()

,2000,2010
California,0.273594,0.249211
New York,0.247010,0.222831
Texas,0.283251,0.273568


## Methods of MultiIndex Creation

계층적 인덱싱을 가진 ``Series``과 ``DataFrame``을 생성하는 방법을 정리해 보자. 

먼저 리스트를 이용한 ``DataFrame`` 생성 예이다.

In [114]:
df = pd.DataFrame(np.random.rand(4, 2),
                  index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                  columns=['data1', 'data2'])
df

data1     data2
a 1  0.733918  0.749057
  2  0.969229  0.199345
b 1  0.787590  0.776047
  2  0.132377  0.286625

dictionary를 이용한 ``Series`` 생성 예이다. 

In [115]:
data = {('California', 2000): 33871648,
        ('California', 2010): 37253956,
        ('Texas', 2000): 20851820,
        ('Texas', 2010): 25145561,
        ('New York', 2000): 18976457,
        ('New York', 2010): 19378102}
pd.Series(data)

California  2000    33871648
            2010    37253956
Texas       2000    20851820
            2010    25145561
New York    2000    18976457
            2010    19378102
dtype: int64

### Explicit MultiIndex constructors

명시적으로  ``MultiIndex`` object를 생성하여 활용하는 게 나을 때도 있다. 

In [116]:
pd.MultiIndex.from_arrays([['a', 'a', 'b', 'b'], [1, 2, 1, 2]])

MultiIndex(levels=[['a', 'b'], [1, 2]],
           codes=[[0, 0, 1, 1], [0, 1, 0, 1]])

In [117]:
pd.MultiIndex.from_tuples([('a', 1), ('a', 2), ('b', 1), ('b', 2)])

MultiIndex(levels=[['a', 'b'], [1, 2]],
           codes=[[0, 0, 1, 1], [0, 1, 0, 1]])

In [118]:
pd.MultiIndex.from_product([['a', 'b'], [1, 2]])

MultiIndex(levels=[['a', 'b'], [1, 2]],
           codes=[[0, 0, 1, 1], [0, 1, 0, 1]])

In [119]:
pd.MultiIndex(levels=[['a', 'b'], [1, 2]],
              labels=[[0, 0, 1, 1], [0, 1, 0, 1]])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning: the 'labels' keyword is deprecated, use 'codes' instead
  


MultiIndex(levels=[['a', 'b'], [1, 2]],
           codes=[[0, 0, 1, 1], [0, 1, 0, 1]])

### MultiIndex level names
``MultiIndex``의 level에 이름을 지어주면 편리한데 ``names`` 속성을 이용하면 된다. 

In [120]:
pop.index.names = ['state', 'year']
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

### MultiIndex for columns

``DataFrame``에서는 rows뿐만 아니라 (당연히) columns도 ``Multiindex``로 인덱싱할 수 있다. 

In [121]:
# hierarchical indices and columns
index = pd.MultiIndex.from_product([[2013, 2014], [1, 2]],
                                   names=['year', 'visit'])
columns = pd.MultiIndex.from_product([['Bob', 'Guido', 'Sue'], ['HR', 'Temp']],
                                     names=['subject', 'type'])

# mock some data
data = np.round(np.random.randn(4, 6), 1)
data[:, ::2] *= 10
data += 37

# create the DataFrame
health_data = pd.DataFrame(data, index=index, columns=columns)
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      41.0  37.5  47.0  37.0  50.0  35.8
     2      50.0  37.5  51.0  38.2  42.0  37.2
2014 1      41.0  37.6  34.0  36.7  45.0  39.1
     2      46.0  36.2  37.0  37.5  30.0  38.5

결과적으로 4차원 데이터가 생성되었다. 


In [122]:
health_data['Guido']

type          HR  Temp
year visit            
2013 1      47.0  37.0
     2      51.0  38.2
2014 1      34.0  36.7
     2      37.0  37.5

## Indexing and Slicing a MultiIndex



### Multiply indexed Series



In [123]:
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [124]:
pop['California', 2000]

33871648

In [125]:
pop['California']

year
2000    33871648
2010    37253956
dtype: int64

In [126]:
pop.loc['California':'New York']

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
dtype: int64

In [127]:
pop[:, 2000]

state
California    33871648
New York      18976457
Texas         20851820
dtype: int64

In [128]:
pop[pop > 22000000]

state       year
California  2000    33871648
            2010    37253956
Texas       2010    25145561
dtype: int64

In [129]:
pop[['California', 'Texas']]

state       year
California  2000    33871648
            2010    37253956
Texas       2000    20851820
            2010    25145561
dtype: int64

### Multiply indexed DataFrames


In [130]:
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      41.0  37.5  47.0  37.0  50.0  35.8
     2      50.0  37.5  51.0  38.2  42.0  37.2
2014 1      41.0  37.6  34.0  36.7  45.0  39.1
     2      46.0  36.2  37.0  37.5  30.0  38.5

``DataFrame``에서는 column이 우선적임을 기억하자. 

In [131]:
health_data['Guido', 'HR']

year  visit
2013  1        47.0
      2        51.0
2014  1        34.0
      2        37.0
Name: (Guido, HR), dtype: float64

In [132]:
health_data.iloc[:2, :2]

subject      Bob      
type          HR  Temp
year visit            
2013 1      41.0  37.5
     2      50.0  37.5

row또는 column에서 계층인덱싱을 할 때에는 tuple을 사용한다. 

In [133]:
health_data.loc[:, ('Bob', 'HR')]

year  visit
2013  1        41.0
      2        50.0
2014  1        41.0
      2        46.0
Name: (Bob, HR), dtype: float64

안타깝게도 tuple내 slicing은 문법오류이다. 

In [134]:
health_data.loc[(:, 1), (:, 'HR')]

SyntaxError: ignored

이러한 처리를 위해서는 다음과 같이 Pandas ``IndexSlice`` object를 이용하여 처리할 수 있다. 

In [136]:
idx = pd.IndexSlice
health_data.loc[idx[:, 1], idx[:, 'HR']]

,subject,Bob,Guido,Sue
,type,HR,HR,HR
year,visit,,,
2013,1,41.0,47.0,50.0
2014,1,41.0,34.0,45.0


## Rearranging Multi-Indices

``stack()`` 과 ``unstack()`` method처럼 데이터를 재배치하는 경우가 있다. 

### Sorted and unsorted indices

많은 ``MultiIndex`` slicing 연산들은 index가 정렬(sorted)되어 있어야만 작동함을 기억하자.

In [137]:
index = pd.MultiIndex.from_product([['a', 'c', 'b'], [1, 2]])
data = pd.Series(np.random.rand(6), index=index)
data.index.names = ['char', 'int']
data

char  int
a     1      0.046987
      2      0.164665
c     1      0.239406
      2      0.153566
b     1      0.943051
      2      0.154873
dtype: float64

현재상태에서 slice를 시도하면 에러가 발생한다. 

In [138]:
try:
    data['a':'b']
except KeyError as e:
    print(type(e))
    print(e)

<class 'pandas.errors.UnsortedIndexError'>
'Key length (1) was greater than MultiIndex lexsort depth (0)'


따라서 인덱스 정렬되어 있지 않다면  ``DataFrame``의 ``sort_index()`` 혹은 ``sortlevel()`` method를 통해 정렬을 먼저 해야 한다. 

In [151]:
data = data.sort_index()
data

char  int
a     1      0.046987
      2      0.164665
b     1      0.943051
      2      0.154873
c     1      0.239406
      2      0.153566
dtype: float64

In [140]:
data['a':'b']

char  int
a     1      0.046987
      2      0.164665
b     1      0.943051
      2      0.154873
dtype: float64

### Stacking and unstacking indices



In [141]:
pop.unstack(level=0)

state,California,New York,Texas
year,,,
2000,33871648,18976457,20851820
2010,37253956,19378102,25145561


In [142]:
pop.unstack(level=1)

year,2000,2010
state,,
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,25145561


In [143]:
pop.unstack().stack()

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

### Index setting and resetting

``reset_index`` method를 통해 index들을 column으로 변환할 수 있다. 

In [144]:
pop_flat = pop.reset_index(name='population')
pop_flat

,state,year,population
0,California,2000,33871648
1,California,2010,37253956
2,New York,2000,18976457
3,New York,2010,19378102
4,Texas,2000,20851820
5,Texas,2010,25145561


반대로 ``set_index`` method를 통해 column을 index로 바꿀 수 있다.

In [145]:
pop_flat.set_index(['state', 'year'])

population
state      year            
California 2000    33871648
           2010    37253956
New York   2000    18976457
           2010    19378102
Texas      2000    20851820
           2010    25145561

## Data Aggregations on Multi-Indices

``mean()``, ``sum()``,  ``max()``와 같은 aggregation 함수에 대해 계층적 인덱싱된 data를 적용할 경우 ``level`` 속성을 지정하면 된다. 

In [146]:
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      41.0  37.5  47.0  37.0  50.0  35.8
     2      50.0  37.5  51.0  38.2  42.0  37.2
2014 1      41.0  37.6  34.0  36.7  45.0  39.1
     2      46.0  36.2  37.0  37.5  30.0  38.5

In [147]:
data_mean = health_data.mean(level='year')
data_mean

subject   Bob       Guido         Sue      
type       HR  Temp    HR  Temp    HR  Temp
year                                       
2013     45.5  37.5  49.0  37.6  46.0  36.5
2014     43.5  36.9  35.5  37.1  37.5  38.8

뒤에서 배울 ``GroupBy`` 기능에서 다시 다룬다. 